In [35]:
import requests
import pandas as pd 
import json
from dateutil import parser

In [4]:
API_KEY = "dffb9bf61baae2cbe74001e07a1f0028-008bb9ba3aff35ad2f21bc9ca89b2c00"
ACCOUNT_ID = "101-001-27747057-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [5]:
session = requests.Session()

In [6]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [7]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [8]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [9]:
response = session.get(url, params=None, data=None, headers=None)

In [10]:
response.status_code

200

In [11]:
data = response.json()

In [12]:
instruments_list = data["instruments"]

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize','marginRate']

In [15]:
instrument_dict = {}
for i in instruments_list:
    key = i["name"]
    instrument_dict[key] = {k: i[k] for k in key_i}

In [16]:
instrument_dict

{'USD_HKD': {'name': 'USD_HKD',
  'type': 'CURRENCY',
  'displayName': 'USD/HKD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.1'},
 'USD_CNH': {'name': 'USD_CNH',
  'type': 'CURRENCY',
  'displayName': 'USD/CNH',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.05'},
 'SGD_CHF': {'name': 'SGD_CHF',
  'type': 'CURRENCY',
  'displayName': 'SGD/CHF',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.05'},
 'NZD_CAD': {'name': 'NZD_CAD',
  'type': 'CURRENCY',
  'displayName': 'NZD/CAD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'minimumTradeSize': '1',
  'marginRate': '0.03'},
 'NZD_JPY': {'name': 'NZD_JPY',
  'type': 'CURRENCY',
  'displayName': 'NZD/JPY',
  'pipLocation': -2,
  'displayPrecision': 3,
  'tradeUnitsPrecision': 0,
  'minimumTra

In [32]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instrument_dict, indent=2))

In [54]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if "candles" not in data:
            data = []
        else:
            data = data["candles"]
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]
    
    final_data = []

    for candle in data:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = parser.parse(candle["time"])
        new_dict["volume"] = candle["volume"]
        for  p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])

        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair, count=10, granularity="H4"):
    code, data = fetch_candles(pair, count=count, granularity=granularity)
    if code != 200:
        print("Failed", pair, data)
        return 
    if len(data) == 0:
        print("no candles", pair)
    candle_df = get_candles_df(data)
    candle_df.to_pickle(f"../data/{pair}_{granularity}.pkl")
    print(f"{pair} {granularity} {candle_df.shape[0]} candles {candle_df.time.min()} {candle_df.time.max()}")
    


In [55]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candle_df = get_candles_df(data)

In [56]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 10 candles 2025-05-08 05:00:00+00:00 2025-05-09 17:00:00+00:00


In [57]:
our_currencies = ["EUR", "USD", "GBP", "JPY", "CHF", "NZD", "CAD", "AUD"]

In [59]:
for p1 in our_currencies:
    for p2 in our_currencies:
        pair = f"{p1}_{p2}"
        if pair in instrument_dict:
            for g in ["H1", "H4"]:
                create_data_file(pair, count=4001, granularity=g)

EUR_USD H1 4001 candles 2024-09-17 04:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_USD H4 4001 candles 2022-10-13 01:00:00+00:00 2025-05-09 17:00:00+00:00
EUR_GBP H1 4001 candles 2024-09-17 04:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_GBP H4 4001 candles 2022-10-13 01:00:00+00:00 2025-05-09 17:00:00+00:00
EUR_JPY H1 4001 candles 2024-09-17 04:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_JPY H4 4001 candles 2022-10-13 01:00:00+00:00 2025-05-09 17:00:00+00:00
EUR_CHF H1 4001 candles 2024-09-17 04:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_CHF H4 4001 candles 2022-10-13 01:00:00+00:00 2025-05-09 17:00:00+00:00
EUR_NZD H1 4001 candles 2024-09-17 03:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_NZD H4 4001 candles 2022-10-13 09:00:00+00:00 2025-05-09 17:00:00+00:00
EUR_CAD H1 4001 candles 2024-09-17 04:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_CAD H4 4001 candles 2022-10-13 01:00:00+00:00 2025-05-09 17:00:00+00:00
EUR_AUD H1 4001 candles 2024-09-17 04:00:00+00:00 2025-05-09 20:00:00+00:00
EUR_AUD H4 4